# GMail

本加载器介绍如何从GMail加载数据。从GMail加载数据有很多方法。该加载器目前在如何实现这一点上相当有主见。它的工作方式是首先查找您发送的所有消息。然后，它查找您回复之前电子邮件的消息。接着，它获取那封之前的电子邮件，并创建一个训练示例，包括那封电子邮件，后面跟着您的电子邮件。

请注意，这里有明显的限制。例如，创建的所有示例只查看前一封电子邮件作为上下文。

使用步骤：

- 设置Google开发者账户：前往Google开发者控制台，创建一个项目，并为该项目启用Gmail API。这将为您提供以后需要的credentials.json文件。

- 安装Google客户端库：运行以下命令安装Google客户端库：

In [ ]:
%pip install --upgrade --quiet  google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [6]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]


creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("email_token.json"):
    creds = Credentials.from_authorized_user_file("email_token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            # your creds file here. Please create json file as here https://cloud.google.com/docs/authentication/getting-started
            "creds.json",
            SCOPES,
        )
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("email_token.json", "w") as token:
        token.write(creds.to_json())

In [7]:
from langchain_community.chat_loaders.gmail import GMailLoader

In [10]:
loader = GMailLoader(creds=creds, n=3)

In [11]:
data = loader.load()

In [13]:
# Sometimes there can be errors which we silently ignore
len(data)

2

In [14]:
from langchain_community.chat_loaders.utils import (
    map_ai_messages,
)

In [17]:
# This makes messages sent by hchase@langchain.com the AI Messages
# This means you will train an LLM to predict as if it's responding as hchase
training_data = list(
    map_ai_messages(data, sender="Harrison Chase <hchase@langchain.com>")
)